# Intergration of TDP GPS Data 



In [16]:
# Environment setup
import pandas as pd
from sqlalchemy import create_engine, text
import logging
import traceback
import os
from sqlalchemy import create_engine
import geopandas as gpd
from datetime import datetime
import configparser
from src.data_ingestion.ingestion_utils import (
    get_db_connection, update_tracking, load_processed_files
)
from src.data_ingestion.eqb_ppData import process_pluspro_data
from src.data_ingestion.eqb_resultsCharts import process_resultscharts_data
from src.data_ingestion.tpd_datasets import (
    process_tpd_sectionals_data,
    process_tpd_gpsdata_data
)

# Load the configuration file
config = configparser.ConfigParser()
config.read('/home/exx/myCode/horse-racing/FoxRiverAIRacing/config.ini')

# Set up logging for consistent logging behavior in Notebook
logging.basicConfig(level=logging.INFO)

# Retrieve database credentials from config file
# Retrieve database credentials from config file
db_host = config['database']['host']
db_port = config['database']['port']
db_name = config['database']['dbname']  # Corrected from 'name' to 'dbname'
db_user = config['database']['user']

# Establish connection using get_db_connection
conn = get_db_connection(config)

# Create the SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{db_user}@{db_host}:{db_port}/{db_name}')

In [24]:
query = """
CREATE TABLE gps_aggregated_results AS
WITH gps_aggregated AS (
    SELECT
        gp.course_cd,
        gp.race_date,
        gp.race_number,
        gp.saddle_cloth_number,
        AVG(gp.speed) AS avg_speed,
        MAX(gp.speed) AS max_speed,
        MIN(gp.speed) AS min_speed,
        AVG(acceleration) AS avg_acceleration,
        MAX(acceleration) AS max_acceleration,
        AVG(gp.stride_frequency) AS avg_stride_freq,
        MAX(gp.stride_frequency) AS max_stride_freq
    FROM (
        SELECT
            gp.*,
            (gp.speed - LAG(gp.speed) OVER w) / NULLIF(EXTRACT(EPOCH FROM (gp.time_stamp - LAG(gp.time_stamp) OVER w)), 0) AS acceleration
        FROM
            gpspoint gp
        WINDOW
            w AS (
                PARTITION BY gp.course_cd, gp.race_date, gp.race_number, gp.saddle_cloth_number
                ORDER BY gp.time_stamp
            )
    ) gp
    GROUP BY
        gp.course_cd,
        gp.race_date,
        gp.race_number,
        gp.saddle_cloth_number
)
SELECT * FROM gps_aggregated;

"""

try:
    with engine.begin() as connection:
        connection.execute(text(query))
        logging.info("Data inserted/updated successfully.")
except Exception as e:
    print(f"An error occurred: {e}")
    traceback.print_exc()

In [27]:
query = """
select * from gps_aggregated_results 
"""

df = pd.read_sql_query(query, engine)

# Display the DataFrame
print(len(df))


477685


## Overview

### Objective:

1.	Aggregate sectionals Data: Create a sectionals_aggregated table with meaningful features.
2.	Join Aggregated Tables: Combine gps_aggregated_results and sectionals_aggregated into a tpd_features table.
3.	Verify and Inspect the Data: Ensure the integration is successful.
4.	Proceed with Data Analysis and Feature Engineering: Prepare the data for modeling with XGBoost.

> Step 1: Aggregate sectionals Data

>> First, we’ll create an aggregated view of your sectionals data. This aggregation will compute various pace-related metrics and stride information for each horse in each race.

>> 1.1. Identify Unique gate_name Values

>> Before writing the aggregation query, it’s crucial to understand the unique gate_name values in your sectionals table. This ensures that our categorization of early and late pace sections is accurate.

> SQL Query to List Unique gate_name Values:

In [18]:
query = """
SELECT DISTINCT gate_name FROM sectionals ORDER BY gate_name;
"""
df = pd.read_sql_query(query, engine)

# Display the DataFrame
print (df)


    gate_name
0       0.18f
1       0.32f
2        0.5f
3        0.5F
4       0.68f
..        ...
125      9.5f
126      9.5F
127        9f
128        9F
129    Finish

[130 rows x 1 columns]


In [28]:
# Define your SQL aggregation query
sectionals_aggregated_query = """
CREATE TABLE sectionals_aggregated AS
WITH ranked_sectionals AS (
    SELECT
        s.course_cd,
        s.race_date,
        s.race_number,
        s.saddle_cloth_number,
        s.gate_name,
        s.length_to_finish,
        s.sectional_time,
        s.running_time,
        s.distance_back,
        s.distance_ran,
        s.number_of_strides,
        s.post_time,
        ROW_NUMBER() OVER (
            PARTITION BY s.course_cd, s.race_date, s.race_number, s.saddle_cloth_number
            ORDER BY s.length_to_finish DESC
        ) AS section_rank,
        COUNT(*) OVER (
            PARTITION BY s.course_cd, s.race_date, s.race_number, s.saddle_cloth_number
        ) AS total_sections
    FROM
        sectionals s
),
categorized_sectionals AS (
    SELECT
        rs.*,
        CASE 
            WHEN rs.section_rank <= CEIL(rs.total_sections * 0.25) THEN 'early'
            WHEN rs.section_rank > rs.total_sections - FLOOR(rs.total_sections * 0.25) THEN 'late'
            ELSE 'middle'
        END AS pace_category
    FROM
        ranked_sectionals rs
)
SELECT
    cs.course_cd,
    cs.race_date,
    cs.race_number,
    cs.saddle_cloth_number,
    SUM(CASE WHEN cs.pace_category = 'early' THEN cs.sectional_time ELSE 0 END) AS early_pace_time,
    SUM(CASE WHEN cs.pace_category = 'late' THEN cs.sectional_time ELSE 0 END) AS late_pace_time,
    SUM(cs.sectional_time) AS total_race_time,
    (SUM(CASE WHEN cs.pace_category = 'early' THEN cs.sectional_time ELSE 0 END) - 
     SUM(CASE WHEN cs.pace_category = 'late' THEN cs.sectional_time ELSE 0 END)) AS pace_differential,
    SUM(cs.number_of_strides) AS total_strides,
    SUM(cs.distance_ran) / NULLIF(SUM(cs.number_of_strides), 0) AS avg_stride_length
FROM
    categorized_sectionals cs
GROUP BY
    cs.course_cd,
    cs.race_date,
    cs.race_number,
    cs.saddle_cloth_number;
"""

try:
    with engine.begin() as connection:
        connection.execute(text(f"DROP TABLE IF EXISTS sectionals_aggregated;"))
        connection.execute(text(sectionals_aggregated_query))
        logging.info("Sectionals aggregated successfully.")
except Exception as e:
    print(f"An error occurred: {e}")
    traceback.print_exc()

In [29]:
# Define your SQL verification query
verify_sectionals_query = "SELECT * FROM sectionals_aggregated LIMIT 10;"

try:
    # Read the data into a pandas DataFrame
    df_sectionals_aggregated = pd.read_sql_query(verify_sectionals_query, engine)
    
    # Display the DataFrame
    print(df_sectionals_aggregated.head())
except Exception as e:
    print(f"An error occurred while verifying sectionals_aggregated: {e}")
    traceback.print_exc()

  course_cd   race_date  race_number saddle_cloth_number  early_pace_time  \
0       AQU  2022-12-29            1                 01             24.04   
1       AQU  2022-12-29            1                 02             23.38   
2       AQU  2022-12-29            1                 03             23.49   
3       AQU  2022-12-29            1                 04             23.84   
4       AQU  2022-12-29            1                 05             24.34   

   late_pace_time  total_race_time  pace_differential  total_strides  \
0           19.83            80.39               4.21          194.2   
1           22.07            83.55               1.31          191.5   
2           20.68            81.27               2.81          187.4   
3           22.55            84.64               1.29          201.2   
4           21.97            84.02               2.37          189.0   

   avg_stride_length  
0           6.780639  
1           6.857441  
2           6.966382  
3           

In [30]:
# Define your SQL integration query
tpd_features_query = """
CREATE TABLE tpd_features AS
SELECT
    gp.course_cd,
    gp.race_date,
    gp.race_number,
    gp.saddle_cloth_number,
    gp.avg_speed,
    gp.max_speed,
    gp.min_speed,
    gp.avg_acceleration,
    gp.max_acceleration,
    gp.avg_stride_freq,
    gp.max_stride_freq,
    s.early_pace_time,
    s.late_pace_time,
    s.pace_differential,
    s.total_race_time,
    s.total_strides,
    s.avg_stride_length
FROM
    gps_aggregated_results gp
INNER JOIN
    sectionals_aggregated s 
    ON gp.course_cd = s.course_cd
    AND gp.race_date = s.race_date
    AND gp.race_number = s.race_number
    AND gp.saddle_cloth_number = s.saddle_cloth_number;
"""

try:
    with engine.begin() as connection:
        # Optional: Drop the table if it already exists
        connection.execute(text("DROP TABLE IF EXISTS tpd_features;"))
        
        # Execute the integration query
        connection.execute(text(tpd_features_query))
        
        logging.info("tpd_features table created successfully.")
except Exception as e:
    print(f"An error occurred while creating tpd_features: {e}")
    traceback.print_exc()

In [31]:
# Define your SQL verification query
verify_tpd_features_query = "SELECT * FROM tpd_features LIMIT 10;"

try:
    # Read the data into a pandas DataFrame
    df_tpd_features = pd.read_sql_query(verify_tpd_features_query, engine)
    
    # Display the DataFrame
    print(df_tpd_features.head())
except Exception as e:
    print(f"An error occurred while verifying tpd_features: {e}")
    traceback.print_exc()

  course_cd   race_date  race_number saddle_cloth_number  avg_speed  \
0       AQU  2022-12-29            1                 01   14.580673   
1       AQU  2022-12-29            1                 02   14.098835   
2       AQU  2022-12-29            1                 03   14.377282   
3       AQU  2022-12-29            1                 04   13.776058   
4       AQU  2022-12-29            1                 05   13.972718   

   max_speed  min_speed  avg_acceleration  max_acceleration  avg_stride_freq  \
0      18.04       0.03          0.116990              5.71         2.380105   
1      18.38       0.03          0.105686              4.51         2.259792   
2      18.47       0.00          0.110980              4.72         2.257447   
3      18.24       0.05          0.108252              5.97         2.342043   
4      18.20       0.03          0.102353              4.86         2.215053   

   max_stride_freq  early_pace_time  late_pace_time  pace_differential  \
0             2.62